In [ ]:
import pandas as pd
import geojson
import random
import matplotlib as plt
from geojson import Polygon, FeatureCollection, Feature, MultiPolygon
import folium
from matplotlib.cm import ScalarMappable
import geojsonio

random.seed(2345698)

boxed_baltimore_file = "modified_data/boxed_baltimore.geojson"

In [ ]:
pc = pd.read_pickle("modified_data/clean_parking_citations")

In [ ]:
with open(boxed_baltimore_file, "r") as bb_file:
    boxed_baltimore_geojson_string = bb_file.read()
    boxed_baltimore_geojson = geojson.loads(boxed_baltimore_geojson_string)

In [ ]:
pc.sample(50)

# Choropleth of All Parking Citations Counts by GeoBox

In [ ]:
def getGeoJsonForCountsForSeries(seriesData):
    boxCounts_all = seriesData.value_counts()
    # norm_colors = plt.colors.Normalize(min(boxCounts_all), max(boxCounts_all))
    norm_colors = plt.colors.LogNorm(min(boxCounts_all), max(boxCounts_all))

    scalarMap = ScalarMappable(norm=norm_colors, cmap="viridis")

    featureList = boxed_baltimore_geojson.features.copy()
    for i in range(0, 625):
        c = boxCounts_all.get(i, -1)
        feature = featureList[i]
        if c == -1:
            feature.properties["fill-opacity"] = 0.0
            feature.properties["stroke-width"] = 0
        else:
            feature.properties["fill-opacity"] = 0.6
            fillColorArray = scalarMap.to_rgba(c, alpha = 0.6)
            fillColor = "rgb({}, {}, {})".format(int(fillColorArray[0]*100), int(fillColorArray[1]*100), int(fillColorArray[2]*100))
            feature.properties["fill"] = fillColor
            feature.properties["stroke"] = fillColor
            feature.properties["count"] = str(c)
    return FeatureCollection(featureList)

In [ ]:
def foliumMapImageOfGeoJsonCounts(geojsonCounts):
    # geojsonio.display(geojson.dumps(all_parking_tickets_geojson))

    def styleFunction(feature):
        return {
            'fillOpacity': feature["properties"]["fill-opacity"],
            'fillColor': feature["properties"]["fill"],
            'weight': 0
        }

    foliumMap = folium.Map(location=[39.2904, -76.6122], zoom_start=11)
    folium.GeoJson(geojson.dumps(geojsonCounts),
                   name='geojson',
                   style_function = styleFunction
                  ).add_to(foliumMap)
    return foliumMap

In [ ]:
foliumMapImageOfGeoJsonCounts(getGeoJsonForCountsForSeries(pc["GeoBox"]))

# Maps for the Top 5 Descriptions

In [ ]:
top5 = pc["Description"].value_counts()[0:5]

In [ ]:
description = top5.index[0]
print(description)
foliumMapImageOfGeoJsonCounts(getGeoJsonForCountsForSeries(pc[pc["Description"] == description]["GeoBox"]))

In [ ]:
description = top5.index[1]
print(description)
foliumMapImageOfGeoJsonCounts(getGeoJsonForCountsForSeries(pc[pc["Description"] == description]["GeoBox"]))

In [ ]:
description = top5.index[2]
print(description)
foliumMapImageOfGeoJsonCounts(getGeoJsonForCountsForSeries(pc[pc["Description"] == description]["GeoBox"]))

In [ ]:
description = top5.index[3]
print(description)
foliumMapImageOfGeoJsonCounts(getGeoJsonForCountsForSeries(pc[pc["Description"] == description]["GeoBox"]))

In [ ]:
description = top5.index[4]
print(description)
foliumMapImageOfGeoJsonCounts(getGeoJsonForCountsForSeries(pc[pc["Description"] == description]["GeoBox"]))